In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, array_to_img, load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPool2D, Dropout
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from PIL import Image
from tensorflow.keras.optimizers import Adam


%matplotlib inline

In [2]:
data_dir = 'dataset'
train_path = 'dataset/Train'
test_path = 'dataset/Test'
IMG_HEIGHT = 30
IMG_WIDTH = 30
channels = 3
NUM_CATEGORIES = len(os.listdir(train_path))
NUM_CATEGORIES

43

In [3]:
# Label Overview
classes = { 0:'Speed limit (20km/h)',
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

In [4]:
def load_data(data_dir):
   
    images = list()
    labels = list()
    for category in range(NUM_CATEGORIES):
        categories = os.path.join(data_dir, str(category))
        for img in os.listdir(categories):
            img = load_img(os.path.join(categories, img), target_size=(30, 30))
            image = img_to_array(img)
            images.append(image)
            labels.append(category)
    
    return images, labels
        

In [5]:
images, labels = load_data(train_path)

# One hot encoding the labels
labels = to_categorical(labels)

# Splitting the dataset into training and test set
x_train, x_test, y_train, y_test = train_test_split(
                                                    np.array(images),
                                                    labels,
                                                    test_size=0.2 , 
                                                    random_state=42, 
                                                    shuffle=True
                                                    )

In [6]:
print("X_train.shape", x_train.shape)
print("X_valid.shape", x_test.shape)
print("y_train.shape", y_train.shape)
print("y_valid.shape", y_test.shape)

X_train.shape (31367, 30, 30, 3)
X_valid.shape (7842, 30, 30, 3)
y_train.shape (31367, 43)
y_valid.shape (7842, 43)


In [7]:
model = Sequential()
model.add(Conv2D(filters=16, kernel_size=(3,3), activation="relu", input_shape=(IMG_HEIGHT,IMG_WIDTH,3)))
model.add(Conv2D(filters=32, kernel_size=(3,3), activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Conv2D(filters=64, kernel_size=(3,3), activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(rate=0.25))
model.add(Flatten())
model.add(Dense(512, activation="relu"))
model.add(Dropout(rate=0.5))
model.add(Dense(43, activation="softmax"))

In [8]:
imgs_path = "dataset/Train"
data_list = []
labels_list = []
classes_list = 43
for i in range(classes_list):
    i_path = os.path.join(imgs_path, str(i)) #0-42
    for img in os.listdir(i_path):
        im = Image.open(i_path +'/'+ img)
        im = im.resize((30,30))
        im = np.array(im)
        data_list.append(im)
        labels_list.append(i)
data = np.array(data_list)
labels = np.array(labels_list)
print(data)

[[[[186 164 164]
   [172 127 106]
   [172 119  85]
   ...
   [179 122  88]
   [182 130  93]
   [182 127  94]]

  [[173 151 137]
   [160 114  89]
   [166 116  84]
   ...
   [168 122  89]
   [168 126  93]
   [167 124  94]]

  [[178 155 148]
   [176 148 135]
   [181 146 136]
   ...
   [171 158 149]
   [173 159 149]
   [174 161 151]]

  ...

  [[ 91  94 100]
   [ 91  92  95]
   [ 91  94  90]
   ...
   [101 100  96]
   [107 101  98]
   [110 102  97]]

  [[ 99 100 105]
   [ 97  98  98]
   [ 96 100  98]
   ...
   [100 101  94]
   [107 106 102]
   [117 112 107]]

  [[102 105 100]
   [104 106 101]
   [102 102 100]
   ...
   [103 102  98]
   [106 108 104]
   [108 109 106]]]


 [[[133 150  86]
   [122 142  75]
   [118 139  74]
   ...
   [109 119  72]
   [104 109  67]
   [111 111  77]]

  [[117 137  75]
   [123 144  77]
   [125 143  80]
   ...
   [107 119  69]
   [101 108  63]
   [100 102  64]]

  [[108 131  73]
   [114 136  76]
   [114 132  78]
   ...
   [115 125  75]
   [105 117  68]
   [ 99 114

In [9]:
def prep_dataset(X,y):
    X_prep = X.astype('float32')
    y_prep = to_categorical(np.array(y))
    return (X_prep, y_prep)

X, y = prep_dataset(data,labels)

In [10]:
X_train, X_val, Y_train, Y_val = train_test_split(X,y, test_size=0.2, shuffle=True,stratify=y)
X_val, X_test, Y_val, Y_test = train_test_split(X_val,Y_val, test_size=0.5, shuffle=True)

In [11]:
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

history= model.fit(X_train,Y_train,
                 epochs=15,
                 batch_size=64,
                 validation_data=(X_val,Y_val))

Epoch 1/15


2023-12-04 12:55:17.966244: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


491/491 [==============================] - 13s 27ms/step - loss: 1.8826 - accuracy: 0.6184 - val_loss: 0.1574 - val_accuracy: 0.9594
Epoch 2/15
491/491 [==============================] - 13s 27ms/step - loss: 0.3323 - accuracy: 0.9055 - val_loss: 0.0793 - val_accuracy: 0.9804
Epoch 3/15
491/491 [==============================] - 13s 27ms/step - loss: 0.1996 - accuracy: 0.9424 - val_loss: 0.0613 - val_accuracy: 0.9852
Epoch 4/15
491/491 [==============================] - 13s 27ms/step - loss: 0.1537 - accuracy: 0.9562 - val_loss: 0.0378 - val_accuracy: 0.9911
Epoch 5/15
491/491 [==============================] - 14s 28ms/step - loss: 0.1275 - accuracy: 0.9652 - val_loss: 0.0353 - val_accuracy: 0.9898
Epoch 6/15
491/491 [==============================] - 14s 28ms/step - loss: 0.1084 - accuracy: 0.9690 - val_loss: 0.0461 - val_accuracy: 0.9913
Epoch 7/15
491/491 [==============================] - 14s 29ms/step - loss: 0.1006 - accuracy: 0.9713 - val_loss: 0.0308 - val_accuracy: 0.9923
Epo

In [12]:
pred = model.predict(X_test)
pred

array([[9.4302365e-13, 1.6727354e-08, 6.1924277e-07, ..., 2.2777735e-09,
        4.3566710e-13, 2.0242570e-08],
       [1.1368686e-21, 1.7448228e-13, 3.9088585e-18, ..., 2.5544065e-16,
        6.9368546e-21, 8.6073560e-25],
       [1.1106813e-09, 2.3322917e-05, 3.6433339e-05, ..., 1.3217880e-08,
        2.1907909e-09, 2.2781881e-09],
       ...,
       [1.3209550e-14, 1.2064522e-13, 6.2682838e-11, ..., 1.9204329e-13,
        6.8440933e-14, 3.8148649e-15],
       [1.2125979e-21, 1.4483848e-15, 1.3267298e-18, ..., 5.4860543e-23,
        5.5744435e-16, 8.5147910e-13],
       [9.4751779e-24, 1.5865095e-15, 1.0065631e-16, ..., 3.7795247e-21,
        4.9031487e-16, 4.0800463e-16]], dtype=float32)

In [13]:
loss, accuracy = model.evaluate(x_test, y_test)

print('test set accuracy: ', accuracy * 100)

246/246 [==============================] - 1s 5ms/step - loss: 0.0458 - accuracy: 0.9897
test set accuracy:  98.9670991897583


In [18]:
tf.saved_model.save(model, 'Downloads/openSW/OSS_Jetbot_Team7/Final_Project/model')

INFO:tensorflow:Assets written to: Downloads/openSW/OSS_Jetbot_Team7/Final_Project/model/assets


INFO:tensorflow:Assets written to: Downloads/openSW/OSS_Jetbot_Team7/Final_Project/model/assets


In [14]:
model.save("my_h5_model.h5")